# Import Library

必要なライブラリを追加する

In [1]:
import os
import glob
import cv2
import sys
import re
import random
import numpy as np
from PIL import Image, ImageOps

# 0. Folder Structure Arrangement 
前処理を行う前に, 下のようにイメージのフォルダー構造を整える必要がある。directory_A, directory_Bは各種類のフォルダーである。

注意：　#TODOが見つかった時は実際に名前やパスなどを合わせてください

In [2]:
# TODO: 画像にあるメインフォルダーのパスを変えてください　(~/data)
data_path = '/home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant'

# 1. Rename file

In [3]:
if os.path.exists(data_path):
    input_dirname = os.path.join(data_path, '*')
    dirs = [d for d in glob.glob(input_dirname) if os.path.isdir(d)]
    print("Number of folders : " + str(len(dirs)))
        
    for kind, dir in enumerate(dirs):
        print(kind, dir)
        
        files = glob.glob(os.path.join(dir, '*.txt'))
        files.sort()
        base_name = os.path.basename(dir)
        
        files_count = len(files)
        renamed_count = 0
        no_image_count = 0
        
        for num, file in enumerate(files):
            # txtファイルを名前の変更
            dname = os.path.dirname(file)
            tgt_fname = os.path.join(dname, base_name+str(num+1).zfill(3)+'.txt')
            if not os.path.exists(tgt_fname):
                os.rename(file, tgt_fname)
                renamed_count += 1
            
            #　イメージの名前の変更
            file2 = file.replace('.txt', '.jpg')
            if not os.path.exists(file2):
                file2 = file.replace('.txt', '.png')
                if not os.path.exists(file2):
                    file2 = file.replace('.txt', '.jpeg')
                    if not os.path.exists(file2):
                        file2 = file.replace('.txt', '.JPG')
                        if not os.path.exists(file2):
                            no_image_count += 1
                            continue
            tgt_fname2 = tgt_fname.replace('.txt', '.jpg')
            if not os.path.exists(tgt_fname2):
                os.rename(file2, tgt_fname2)
                
        print(">Total files:{}, Renamed files:{}, Error: {}".format(files_count, renamed_count, no_image_count))
else:
    print("Erro:　Path not found!")

Number of folders : 14
0 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant/Dendrobium_Nobile
>Total files:17, Renamed files:0, Error: 0
1 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant/Oncidium
>Total files:50, Renamed files:0, Error: 0
2 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant/Nepenthes
>Total files:17, Renamed files:0, Error: 0
3 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant/Phalaenopsis
>Total files:50, Renamed files:0, Error: 0
4 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant/DragonFruit
>Total files:50, Renamed files:0, Error: 0
5 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant/Vanda
>Total files:50, Renamed files:0, Error: 0
6 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/Plant/Cacao
>Total files:13, Renamed files:0, Error: 0
7 /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/

# 2. Rewrite label (create classes.txt)

In [4]:
# TODO: fish以外場合はファイル名を変えてください
obj_names = "classes.txt"
label_file = open(obj_names, "w")

input_dirname = os.path.join(data_path, '*')
dirs = [d for d in glob.glob(input_dirname) if os.path.isdir(d)]

num_classes = len(dirs)

print("There are {} classes:".format(num_classes))

for kind, dir in enumerate(dirs):
    files = glob.glob(os.path.join(dir, '*.txt'))
    label_name = os.path.basename(dir)
    label_file.write('{}\n'.format(label_name))
    print("{} : {}".format(kind, label_name))
    for file in files:
        with open(file, mode='r') as f:
            lines = f.readlines()
        with open(file, mode='w') as f:
            for line in lines:
                line2 = line.split()
                line2[0] = str(kind)
                f.write('{} {} {} {} {}\n'.format(line2[0], line2[1], line2[2], line2[3], line2[4]))
                
label_file.close()

print("--------------")
print("{} is created at \n{}".format(obj_names, os.getcwd()))

There are 14 classes:
0 : Dendrobium_Nobile
1 : Oncidium
2 : Nepenthes
3 : Phalaenopsis
4 : DragonFruit
5 : Vanda
6 : Cacao
7 : ParaguaiOni_Bus
8 : Sausage_Tree
9 : Bulbophyllum
10 : Cattleya
11 : Hibiscus
12 : Saihai_Deigo
13 : Tropical_Water_Lily
--------------
classes.txt is created at 
/home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens


# 3. Create Augmented Data (Optional)

In [6]:
#TODO augmented no folder wo atarasiku tukuttene
original_dir = data_path  
aug_path = os.path.join(os.path.dirname(data_path), 'augmentedSenriai') 

if not os.path.exists(aug_path):
        os.makedirs(aug_path) 
        
class Augmentation:
    def __init__(self, original_dir, aug_path):
        self.original_dir = original_dir
        self.aug_path = aug_path
    
    # ヒストグラム均一化
    def equalizeHistRGB(self, src):
        RGB = cv2.split(src)
        Blue   = RGB[0]
        Green = RGB[1]
        Red    = RGB[2]
        for i in range(3):
            cv2.equalizeHist(RGB[i])

        img_hist = cv2.merge([RGB[0],RGB[1], RGB[2]])
        return img_hist

    # ガウシアンノイズ
    def addGaussianNoise(self, src):
        row,col,ch= src.shape
        mean = 0
        var = 0.1
        sigma = 15
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = src + gauss

        return noisy

    # salt&pepperノイズ
    def addSaltPepperNoise(self, src):
        row,col,ch = src.shape
        s_vs_p = 0.5
        amount = 0.004
        out = src.copy()
        # Salt mode
        num_salt = np.ceil(amount * src.size * s_vs_p)
        coords = [np.random.randint(0, i-1 , int(num_salt))
                  for i in src.shape]
        out[coords[:-1]] = (255,255,255)

        # Pepper mode
        num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i-1 , int(num_pepper))
                  for i in src.shape]
        out[coords[:-1]] = (0,0,0)
        return out

    def main(self, target_dir, aug_path):
        # ルックアップテーブルの生成
        min_table = 50
        max_table = 205
        diff_table = max_table - min_table
        gamma1 = 0.75
        gamma2 = 1.5

        LUT_HC = np.arange(256, dtype = 'uint8' )
        LUT_LC = np.arange(256, dtype = 'uint8' )
        LUT_G1 = np.arange(256, dtype = 'uint8' )
        LUT_G2 = np.arange(256, dtype = 'uint8' )

        LUTs = []

        # 平滑化用
        average_square = (10,10)

        # ハイコントラストLUT作成
        for i in range(0, min_table):
            LUT_HC[i] = 0

        for i in range(min_table, max_table):
            LUT_HC[i] = 255 * (i - min_table) / diff_table

        for i in range(max_table, 255):
            LUT_HC[i] = 255

        # その他LUT作成
        for i in range(256):
            LUT_LC[i] = min_table + i * (diff_table) / 255
            LUT_G1[i] = 255 * pow(float(i) / 255, 1.0 / gamma1)
            LUT_G2[i] = 255 * pow(float(i) / 255, 1.0 / gamma2)

        LUTs.append(LUT_HC)
        LUTs.append(LUT_LC)
        LUTs.append(LUT_G1)
        LUTs.append(LUT_G2)

        # 画像の読み込み
        files = glob.glob(target_dir+'/*.jpg')

        for item_index, file in enumerate(files):
            img_src = cv2.imread(file, 1)

            trans_img = []
            trans_img.append(img_src)

            # LUT変換
            for i, LUT in enumerate(LUTs):
                trans_img.append( cv2.LUT(img_src, LUT))

            # 平滑化
            trans_img.append(cv2.blur(img_src, average_square))

            # ヒストグラム均一化
            trans_img.append(self.equalizeHistRGB(img_src))

            # ノイズ付加
            trans_img.append(self.addGaussianNoise(img_src))
            trans_img.append(self.addSaltPepperNoise(img_src))

            # 反転
            flip_img = []
            for img in trans_img:
                flip_img.append(cv2.flip(img, 1))
            trans_img.extend(flip_img)

            # 保存
            if not os.path.exists(aug_path):
                os.mkdir(aug_path)

            base = os.path.splitext(os.path.basename(file))[0] + "_"
            item_txt = os.path.splitext(os.path.basename(file))[0] + '.txt'
            img_src.astype(np.float64)
            for i, img in enumerate(trans_img):
                # 比較用
                # cv2.imwrite("target_images/" + base + str(i) + ".jpg" ,cv2.hconcat([img_src.astype(np.float64), img.astype(np.float64)]))
                #9以下は反転していない
                coordinate = ''
                #file = open("target_images/" + item_txt, 'r')
                file = open(target_dir+"/" + item_txt, 'r')
                if i <= 8:
                    coordinate = file.read()
                #9以上は反転している
                else:                    
                    while True:
                        line = file.readline()
                        if not line:
                            break
                        words = re.split(" +", line)
                        if len(words) >= 2:
                            label=words[0]
                            x = words[1]
                            y = words[2]
                            width = words[3]
                            height = words[4]
                            x = str(1.0 - float(x))
                            coordinate = coordinate + ' '.join([
                                str(label).strip(),
                                str(x).strip(),
                                str(y).strip(),
                                str(width).strip(),
                                str(height).strip()
                            ]) + '\n'

                file.close()
                fn = aug_path + "/" + base + str(i) + ".txt"
                with open(fn, 'w') as f:
                    f.write(coordinate)
                cv2.imwrite(aug_path + "/" + base + str(i) + ".jpg" ,img)

    def add_noise(self):
        if not os.path.exists(self.aug_path):
            os.mkdir(self.aug_path)        
        drs=glob.glob(self.original_dir+'/*')
        if os.path.isfile(drs[0]):
            self.main(self.original_dir, self.aug_path)
        else:
            for dr in drs:
                la = os.path.basename(dr)
                inc_sub_dr = os.path.join(self.aug_path, la)
                if not os.path.exists(inc_sub_dr):
                    os.mkdir(inc_sub_dr)
                self.main(dr, inc_sub_dr)

data = Augmentation(original_dir, aug_path)
data.add_noise()

print("Augmented data created")
print("Location: {}".format(aug_path))

<ipython-input-6-98432b8262cf>:47: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords[:-1]] = (255,255,255)
<ipython-input-6-98432b8262cf>:53: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords[:-1]] = (0,0,0)


Augmented data created
Location: /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/augmentedSenriai


# 4. Create Negative Data (Optional)

In [ ]:
# TODO: ネガティブ画像のフォルダーパスを変えてください
negative_input_dirname = '/Users/davidpich/Desktop/negative'

if not os.path.exists(negative_input_dirname):
    print("Error: Folder not found! Please enter a valid path")

negative_basename = os.path.dirname(negative_input_dirname)
negative_output_dirname = os.path.join(negative_basename, 'negative_data')

BaseW = 416
BaseH = 416

reshaped_size = (BaseW, BaseH)

if not os.path.exists(negative_output_dirname):
        os.makedirs(negative_output_dirname) 
files = glob.glob(os.path.join(negative_input_dirname, '*'))

for i, file in enumerate(files):
    index = i + 1
    try:
        image = Image.open(file)
        image.thumbnail(reshaped_size)
        converted = image.convert('RGB')

        save_fl = os.path.join(negative_output_dirname, 'negative'+str(index)+'.jpg')
        save_fl_ano = save_fl.replace('.jpg', '.txt') 
        
        image.save(save_fl)
        
        with open(save_fl_ano, mode='w') as f:
            f.write('')
        
#         """
#         細かい画像を増やしたい場合
#         """
#         #image divider
#         div_x, div_y = converted.size
#         div_x, div_y = div_x//3, div_y//3

#         for k in range(2):
#             for m in range(2):
#                 img = converted.crop((m*div_x, k*div_y, (m+1)*div_x, (k+1)*div_y))
#                 save_fl = nagative_output_dirname+'/'+str(index)+'_'+str(k)+'_'+str(m)+'.jpg'
#                 img.save(save_fl)
#                 save_fl_ano = save_fl.replace('.jpg', '.txt') #アノテーションファイル名
#                 with open(save_fl_ano, mode='w') as f:
#                     f.write('')
        
    except IOError:
        pass
    
print("Negative data created.")
print("Location: {}".format(negative_output_dirname))/augmented

# 5. Split Dataset (Training + Testing)

以下に1つ方法を選んでください (create yolo_train.txt + yolo_test.txt)

### 5.1 Original Data Only

In [12]:
random.seed(42)

dataset_fn = 'dataset.txt'
train_fn = 'yolo_train.txt'
test_fn = 'yolo_test.txt'

dataset_f = open(dataset_fn, 'w')
positive_output_dirname =  data_path

drs=glob.glob(positive_output_dirname+'/*')
drs.sort()
for dr in drs:
    if os.path.isfile(dr):
        continue
    fls = glob.glob(dr+'/*.jpg')
    for fl in fls:
        try:
            dataset_f.write('{}\n'.format(os.path.abspath(fl)))
        except:
            pass
    label_name = os.path.basename(dr)
dataset_f.close()

with open(dataset_fn,mode='r') as f:
    ln = f.readlines()

separate_ratio = 0.1
num_test = (int)(len(ln)*separate_ratio)
num_train = len(ln)-num_test

random.shuffle(ln)
test_data = ln[:num_test]
train_data = ln[num_test:]

with open(train_fn, 'w') as f:
    f.writelines(train_data)
    
with open(test_fn, 'w') as f:
    f.writelines(test_data)

print("Dataset size: " + str(len(ln)))
print("Training data :"+ str(len(train_data)))
print("Validation data :"+ str(len(test_data)))

Dataset size: 1
Training data :1
Validation data :0


### 5.2 Original Data + Negative Data

In [ ]:
random.seed(42)

dataset_fn = 'dataset.txt'
train_fn = 'yolo_train.txt'
test_fn = 'yolo_test.txt'

dataset_f = open(dataset_fn, 'w')
positive_output_dirname =  data_path

drs=glob.glob(positive_output_dirname+'/*')
drs.sort()
for dr in drs:
    if os.path.isfile(dr):
        continue
    fls = glob.glob(dr+'/*.jpg')
    for fl in fls:
        try:
            dataset_f.write('{}\n'.format(os.path.abspath(fl)))
        except:
            pass
    label_name = os.path.basename(dr)
dataset_f.close()

with open(dataset_fn,mode='r') as f:
    ln = f.readlines()

separate_ratio = 0.1
num_test = (int)(len(ln)*separate_ratio)
num_train = len(ln)-num_test

random.shuffle(ln)
test_data = ln[:num_test]
train_data = ln[num_test:]

negative_data = []
files=glob.glob(negative_output_dirname+'/*.jpg')
for i,file in enumerate(files):
    negative_data.append(os.path.abspath(file)+'\n')

random.shuffle(negative_data)

#どれくらいの割合でネガティブデータを利用するか
netative_data_ratio = 1 
negative_data = negative_data[:(int)(len(negative_data)*netative_data_ratio)]

train_data = train_data + negative_data
random.shuffle(train_data)

with open(test_fn, 'w') as f:
    f.writelines(test_data)
with open(train_fn, 'w') as f:
    f.writelines(train_data)
    
print("Dataset size: " + str(len(ln)))
print("Negative data added: " + str(len(negative_data)))
print("Training data :"+ str(len(train_data)))
print("Validation data :"+ str(len(test_data)))

### 5.3 Augmented Data Only

In [7]:
random.seed(42)

dataset_fn = 'dataset.txt'
train_fn = 'yolo_train.txt'
test_fn = 'yolo_test.txt'

dataset_f = open(dataset_fn, 'w')
positive_output_dirname =  aug_path

drs=glob.glob(positive_output_dirname+'/*')
drs.sort()
for dr in drs:
    if os.path.isfile(dr):
        continue
    fls = glob.glob(dr+'/*.jpg')
    for fl in fls:
        try:
            dataset_f.write('{}\n'.format(os.path.abspath(fl)))
        except:
            pass
    label_name = os.path.basename(dr)
dataset_f.close()

with open(dataset_fn,mode='r') as f:
    ln = f.readlines()

separate_ratio = 0.1
num_test = (int)(len(ln)*separate_ratio)
num_train = len(ln)-num_test

random.shuffle(ln)
test_data = ln[:num_test]
train_data = ln[num_test:]

with open(train_fn, 'w') as f:
    f.writelines(train_data)
    
with open(test_fn, 'w') as f:
    f.writelines(test_data)

print("Dataset size: " + str(len(ln)))
print("Training data :"+ str(len(train_data)))
print("Validation data :"+ str(len(test_data)))

Dataset size: 10044
Training data :9040
Validation data :1004


### 5.4 Augmented Data + Negative Data

In [ ]:
random.seed(42)

dataset_fn = 'dataset.txt'
train_fn = 'yolo_train.txt'
test_fn = 'yolo_test.txt'

dataset_f = open(dataset_fn, 'w')
positive_output_dirname =  aug_path

drs=glob.glob(positive_output_dirname+'/*')
drs.sort()
for dr in drs:
    if os.path.isfile(dr):
        continue
    fls = glob.glob(dr+'/*.jpg')
    for fl in fls:
        try:
            dataset_f.write('{}\n'.format(os.path.abspath(fl)))
        except:
            pass
    label_name = os.path.basename(dr)
dataset_f.close()

with open(dataset_fn,mode='r') as f:
    ln = f.readlines()

separate_ratio = 0.1
num_test = (int)(len(ln)*separate_ratio)
num_train = len(ln)-num_test

random.shuffle(ln)
test_data = ln[:num_test]
train_data = ln[num_test:]

negative_data = []
files=glob.glob(negative_output_dirname+'/*.jpg')
for i,file in enumerate(files):
    negative_data.append(os.path.abspath(file)+'\n')

random.shuffle(negative_data)

#どれくらいの割合でネガティブデータを利用するか
netative_data_ratio = 1 
negative_data = negative_data[:(int)(len(negative_data)*netative_data_ratio)]

train_data = train_data + negative_data
random.shuffle(train_data)

with open(test_fn, 'w') as f:
    f.writelines(test_data)
with open(train_fn, 'w') as f:
    f.writelines(train_data)
    
print("Dataset size: " + str(len(ln)))
print("Negative data added: " + str(len(negative_data)))
print("Training data :"+ str(len(train_data)))
print("Validation data :"+ str(len(test_data)))

# 6. Generate data file (create fish.data)


In [8]:
# TODO: fish以外場合はファイル名を変えてください
filename = "senriai.data"
obj_data_path = os.path.join(os.getcwd(), filename)

classes = num_classes
train_path = os.path.join(os.getcwd(), train_fn)
test_path = os.path.join(os.getcwd(), test_fn)
name_path = os.path.join(os.getcwd(), obj_names)
backup = 'backup'
backup_path = os.path.join(os.getcwd(), backup)

if not os.path.exists(backup_path):
    os.makedirs(backup_path)

with open(filename, 'w') as f:
    f.write("classes = {}\ntrain = {}\nvalid = {}\nnames = {}\nbackup ={}".
            format(classes, train_path, test_path, name_path, backup_path))
    
print("{} created".format(filename))
print("Location: {}".format(obj_data_path))

senriai.data created
Location: /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens/senriai.data


# 7. Generate README (create README.txt)

In [9]:
# TODO: 実際のファイル名に合わせてください
import os
filename = "senriai.data"
obj_data_path = os.path.join(os.getcwd(), filename)
backup = 'backup'
pretrained_file = 'darknet53.conv.74'
cfg_file = 'yolov4.cfg'
video_path = 'test1.mp4'
log_file = "log.txt"

backup_path = os.path.join(os.getcwd(), backup)
cfg_path = os.path.join(os.getcwd(), cfg_file)
log_path = os.path.join(os.getcwd(), log_file)
weight_path = os.path.join(backup_path, cfg_file[:-4]+'_last.weight')

training_cmd = "./darknet detector train {} {} {} -map".format(obj_data_path, cfg_path, pretrained_file)
training_str = "[For training]\n{}\n".format(training_cmd)

training_with_log_str = "[For Training with log]\n{} | grep \"avg loss\" | tee {}\n".format(training_cmd, log_path) 

prediction_cmd = "./darknet detector demo {} {} {} {}".format(obj_data_path, cfg_path, weight_path, video_path)
prediction_str = "[For video prediction]\n{}\n".format(prediction_cmd)

evaluation_cmd = "./darknet detector map {} {} {} -iou_thresh 0.75".format(obj_data_path, cfg_path, weight_path)
evaluation_str = "[For evaluation mAP]\n{}\n".format(evaluation_cmd)

with open("README.txt", "w") as f:
    f.write("{}\n{}\n{}\n{}".format(training_str, training_with_log_str, prediction_str, evaluation_str))
    
print("README file created")
print("Location: {}".format(os.getcwd()))

README file created
Location: /home/ic1/darknet/logs/DCON2021_SenriAi/Southeast_Botanical_Gardens
